## B10. Calculate count of VALID_DATA and WATER_MASK and QA bands

#### 0. Load required libraries, site dependant constants and utility functions.

In [2]:
from os import listdir
from os.path import isfile, join
import numpy as np
import seaborn as sns

# EOLearn libraries:
from eolearn.core import EOTask, EOPatch, LinearWorkflow, LoadTask, SaveTask, FeatureType, EOExecutor
from eolearn.core import OverwritePermission

# Add to python path parent dictionary (to have acces to the data and util catalogue)
import sys
sys.path.append("../../")

# load site dependant constants (HERE YOU CAN CHOOSE DIFFERENT LOCATION)
from aoi_sites import upe_promice_area as site

# load utility functions
from utils import io_functions as io_utils
from utils import plot_functions as plot_utils

from scipy import ndimage

#### 1. Create  necessary EOTasks

In [26]:
# define counting mask calculation task:

class CountTruePixelsFromMask(EOTask):   
    """
    The task counts number of valid observations in time-series and stores the results in the timeless mask.
    """
    def __init__(self, count_what, feature_name):
        self.what = count_what
        self.name = feature_name
        
    def execute(self, eopatch):
        eopatch.add_feature(FeatureType.MASK_TIMELESS, self.name, np.count_nonzero(eopatch.mask[self.what],axis=0))
        
        return eopatch
    
class CountTruePixelsFromLandsat8QABands(EOTask):   
    """
    The task counts number of valid observations in time-series and stores the results in the timeless mask.
    """
    def __init__(self, input_data_name , band_idx , output_data_name):
        self.input_data_name = input_data_name
        self.band_idx = band_idx
        self.output_data_name = output_data_name
        
    def execute(self, eopatch):
        data_to_check = eopatch.data[self.input_data_name][..., self.band_idx]
        eopatch.add_feature(FeatureType.MASK_TIMELESS, self.output_data_name, np.count_nonzero(data_to_check,axis=0)[..., np.newaxis])
        
        return eopatch

#### 2. Create and execute EOWorkflow

In [ ]:
landsat_count_valid_eotask = CountTruePixelsFromMask('IS_DATA', 'IS_DATA_COUNT') # name of output scalar
modis_count_valid_eotask = CountTruePixelsFromMask('MODIS_IS_DATA_DAY_0', 'MODIS_IS_DATA_DAY_0_COUNT') # name of output scalar
count_water_eotask = CountTruePixelsFromMask('WATER_MASK_ST_025', 'WATER_MASK_ST_025_COUNT') # name of output scalar
count_cloud_eotask = CountTruePixelsFromMask('CLOUD_MASK_SWIR-1_150_dil_6_str2', 'CLOUD_MASK_SWIR-1_150_dil_6_str2_COUNT') # name of output scalar

count_qa_band_clouds = CountTruePixelsFromLandsat8QABands( input_data_name='LANDSAT_QA_LAYERS' , band_idx=0, output_data_name='LANDSAT_QA_CLOUD_COUNT')
count_qa_band_clouds_and_around = CountTruePixelsFromLandsat8QABands( input_data_name='LANDSAT_QA_LAYERS' , band_idx=1, output_data_name='LANDSAT_QA_CLOUD_AND_AROUND_COUNT')
count_qa_band_clouds_shadow = CountTruePixelsFromLandsat8QABands( input_data_name='LANDSAT_QA_LAYERS' , band_idx=2, output_data_name='LANDSAT_QA_CLOUDS_SHADOW_COUNT')
count_qa_band_cirrus = CountTruePixelsFromLandsat8QABands( input_data_name='LANDSAT_QA_LAYERS' , band_idx=3, output_data_name='LANDSAT_QA_CIRRUS_COUNT')
count_qa_band_snow_ice = CountTruePixelsFromLandsat8QABands( input_data_name='LANDSAT_QA_LAYERS' , band_idx=4, output_data_name='LANDSAT_QA_SNOW_ICE_COUNT')

# application of all eotasks for all eopatches in the a eoworkflow
for date_range_landsat in site.DATE_RANGES_LANDSAT_8 :
    print('Reading EOPatches from:', date_range_landsat)
    eopatches_filepath = io_utils.get_eopatches_dir(site.DATA_PRODUCTS[1], site.SITE_NAME, site.AVAILABLE_CRSs[0], date_range_landsat)
    # TASK TO LOAD AND SAVE EXISTING EOPATCHES HAS TO BE CREATED YEARLY (different directories)
    load_eotask = LoadTask(eopatches_filepath)
    save_eotask = SaveTask(eopatches_filepath, overwrite_permission=OverwritePermission.OVERWRITE_PATCH, compress_level=1)
    
    # Define the workflow
    workflow = LinearWorkflow(
    load_eotask,
    landsat_count_valid_eotask,
    modis_count_valid_eotask,
    count_water_eotask,
    count_cloud_eotask,
    count_qa_band_clouds,
    count_qa_band_clouds_and_around,
    count_qa_band_clouds_shadow,
    count_qa_band_cirrus,
    count_qa_band_snow_ice,
    save_eotask
    )
                      
    list_of_available_patches = io_utils.get_list_of_eopatches(eopatches_filepath)
    execution_args = []
    for eopatch_name in list_of_available_patches:
        execution_args.append({
        load_eotask: {'eopatch_folder': eopatch_name},
        save_eotask: {'eopatch_folder': eopatch_name}
        })
    
    #print(execution_args)                            
    executor = EOExecutor(workflow, execution_args, save_logs=True)
    executor.run(workers=1, multiprocess=False)

    executor.make_report()


Reading EOPatches from: ('2013-05-01', '2013-10-31')



Reading EOPatches from: ('2014-05-01', '2014-10-31')



Reading EOPatches from: ('2015-05-01', '2015-10-31')



Reading EOPatches from: ('2016-05-01', '2016-10-31')



Reading EOPatches from: ('2017-05-01', '2017-10-31')



Reading EOPatches from: ('2018-05-01', '2018-10-31')


In [19]:
# Let's visualize it
workflow.dependency_graph()

#### 3. Visualize data
Plot the map of valid pixel counts

In [24]:
eo_patch = EOPatch.load('../../data/EOPatches/LANDSAT_8/UPE_PROMICE/UTM_22N/2013_05-2013_10/2969_maxcc_0.05_x-3_y-130')
eo_patch

EOPatch(
  data: {
    LANDSAT_EUC_NORM_RAW_BANDS: numpy.ndarray(shape=(7, 333, 333, 1), dtype=float32)
    LANDSAT_NDWI: numpy.ndarray(shape=(7, 333, 333, 1), dtype=float32)
    LANDSAT_NDWI_GREEN_NIR: numpy.ndarray(shape=(7, 333, 333, 1), dtype=float32)
    LANDSAT_NDWI_ICE: numpy.ndarray(shape=(7, 333, 333, 1), dtype=float32)
    LANDSAT_NDWI_NIR_SWIR_1: numpy.ndarray(shape=(7, 333, 333, 1), dtype=float32)
    LANDSAT_NDWI_NIR_SWIR_2: numpy.ndarray(shape=(7, 333, 333, 1), dtype=float32)
    LANDSAT_QA_LAYERS: numpy.ndarray(shape=(7, 333, 333, 5), dtype=float32)
    LANDSAT_RAW_BANDS: numpy.ndarray(shape=(7, 333, 333, 11), dtype=float32)
    MODIS_EUC_NORM_RAW_BANDS: numpy.ndarray(shape=(7, 333, 333, 1), dtype=float32)
    MODIS_NDWI: numpy.ndarray(shape=(7, 333, 333, 1), dtype=float32)
    MODIS_NRB: numpy.ndarray(shape=(7, 333, 333, 1), dtype=float32)
    MODIS_RAW_BANDS_DAY_0: numpy.ndarray(shape=(7, 333, 333, 7), dtype=float32)
    MODIS_RAW_BANDS_DAY_AFTER: numpy.ndarray(shape=(